In [18]:

import os
import glob
from pathlib import Path

import numpy as np
from tqdm import tqdm



base_folder_path = Path('/data/storage025/wavs_single_channel_normalized_nosil/')

np.set_printoptions(threshold=np.inf)


# 3 sublists for YA OA PD
exp2lists = {'BoundaryTone': [[], [], []], 'EarlyLate': [[], [], []], 'PictureNaming': [[], [], []]}
all3 = [[], [], []]


def get_group_id(filename):
    filename = os.path.basename(filename)
    group_id = filename.split('_')[0][-4:-2]
    if group_id not in ['11', '21', '22']:
        raise ValueError(f"Invalid group id {group_id}")
    return group_id

def add2list(group_id, feature, ls):
    if group_id == '11':
        ls[0].append(feature)
    elif group_id == '21':
        ls[1].append(feature)
    elif group_id == '22':
        ls[2].append(feature)
    else:
        print(f'Invalid group id {group_id}')
        
        
        
feature_name = 'f0'        
avg_diff = []
f0_diff_list = open('f0_diff_list.txt', 'w')
non_f0_list = open('non_f0_list.txt', 'w')
for folder in ['BoundaryTone', 'EarlyLate', 'PictureNaming']:
# for folder in ['BoundaryTone']:
    feature_folder = os.path.join(base_folder_path, folder + '-features', feature_name)
    feature_folder = Path(feature_folder)
    npy_files = list(feature_folder.glob('*.npy'))
    print(f'Processing {folder} folder...')
    print(f'Found {len(npy_files)} npy files')
    

    cnt = 0
    cnt2 = 0
    for npy_file in npy_files:
        feature = np.load(npy_file)
        feature_nonorm = np.load(str(npy_file).replace('_normalized_', '_'))
        avg_diff_each = np.average(np.abs(feature - feature_nonorm))
        if avg_diff_each > 1:
            f0_diff_list.write(f'{os.path.basename(npy_file)}\t{avg_diff_each}\n') 
        avg_diff.append(avg_diff_each)
        
        # check if all 0 value
        if np.max(feature) == 0 and np.min(feature) == 0:
            cnt += 1
            non_f0_list.write(f'{os.path.basename(npy_file)}\n')
            continue
        
        if np.max(feature) < 60:
            print(f'max value < 60 in {npy_file}')
            cnt2 += 1
            non_f0_list.write(f'{os.path.basename(npy_file)}\n')

            continue
        
        group_id = get_group_id(npy_file.stem)
        add2list(group_id, feature, exp2lists[folder])
    print(f'{cnt} files with all 0 values')
    print(f'{cnt2} files with max value < 60Hz')


#merge 3lists to all3
for i in range(3):
    all3[i] += exp2lists['BoundaryTone'][i] + exp2lists['EarlyLate'][i] + exp2lists['PictureNaming'][i]

avg_diff = np.array(avg_diff)
# print top 10 largest diff
print(len(avg_diff))
print(np.sort(avg_diff)[-800])
print(np.min(avg_diff))
print(np.average(avg_diff)) 
# close
f0_diff_list.close()
non_f0_list.close()
        


Processing BoundaryTone folder...
Found 2790 npy files
5 files with all 0 values
0 files with max value < 60Hz
Processing EarlyLate folder...
Found 4011 npy files
40 files with all 0 values
0 files with max value < 60Hz
Processing PictureNaming folder...
Found 1792 npy files
14 files with all 0 values
0 files with max value < 60Hz
8593
1.0331181904879838
0.0
0.6633160884302535


In [36]:
# deep copy BD
all3_cp = [[], [], []]
group_id = ['YA', 'OA', 'PD']

for i in range(3):
    tmp = np.concatenate(all3[i], axis=0)
    # remove all 0 value
    print(tmp.shape)
    tmp = tmp[tmp != 0]    
    print(tmp.shape)
    # remove value > 500
    tmp = tmp[tmp < 500.0]
    print(tmp.shape)
    all3_cp[i] = tmp


    
for i, subgroup in enumerate(all3_cp):
    print(f'\n  stats of {group_id[i]}')
    # merge all data in subgroup
    print(subgroup.shape)
    # calculate min, max, mean, std
    print('min:', np.min(subgroup))
    print('max:', np.max(subgroup))
    print('mean:', np.mean(subgroup))
    print('std:', np.std(subgroup))
    print('median:', np.median(subgroup))
    print('25 percentile:', np.percentile(subgroup, 25))
    print('75 percentile:', np.percentile(subgroup, 75))


print('min of max:', minofmax)
print(len(all3[0] + all3[1] + all3[2]))

(272808,)
(163004,)
(162990,)
(393653,)
(224885,)
(224668,)
(283649,)
(136326,)
(136289,)

  stats of YA


AttributeError: 'list' object has no attribute 'shape'